In [ ]:
### import #########################
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()

import numpy as np
import pandas as pd

tf.get_logger().setLevel('ERROR')
import tensorflow_privacy
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
import tensorflow.keras.backend as kb
import h5py

#### Adult Data

In [ ]:
############ ADULT ################
# df = pd.read_csv('adult_prot.csv')
# df_ap = df.loc[(df['103']== 1) & (df['104'] == 1)]
# df=df.append([df_ap]*1)
# df = df[:-611]
# data = df.values[:,:-1]
# prot = df.values[:,-1]
# print(data.shape, prot.shape)
# feat_size = data.shape[1]-1

# df = pd.read_csv('compass_new.csv')
# data = df.values[:,:-1]
# prot = df.values[:, -1]
# feat_size = data.shape[1]-1
# print(sum(prot), sum(data[:,-1]))
# print(feat_size)

#### Bank Data

In [ ]:
# ########## BANK #####################
# df = pd.read_csv('bank_prot.csv')
# indx = df[df['63'] == 0].index.values[:1188]
# update_df = df.drop(indx)

# data = update_df.values[:,:-1]
# prot = update_df.values[:,-1]
# print(sum(prot), data.shape)
# feat_size = data.shape[1] - 1
# print(sum(1 - data[:,-1]))

### Hyperparameters

In [ ]:
# fed_ml params
splits = 5
samples_per_network = int(0.8*len(data)/(splits))
models = []
data_split = []

# architecture
# hidden_size = [1000, 500] ## adult
hidden_size = [500, 100]  ## bank

num_classes = 2
input_size = [samples_per_network, data.shape[1]-1]

# optimizer and loss
learning_rate = 0.25 ## for phase 2
rho = 10
batch_size = 500
# num_epochs = 100 ## phase 1
num_epochs = 20 ## phase 2

l2_norm_clip = 1.5
noise_multiplier = 1.3  ### sigma
num_microbatches = 250

# if batch_size % num_microbatches != 0:
#   raise ValueError('Batch size should be an integer multiple of the number of microbatches')

# fairness params
epsilon = 0.01

#### MODEL

In [ ]:
def model_init(flag):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(hidden_size[0],
                               activation='relu',
                               input_shape=(feat_size,)),
        tf.keras.layers.Dense(hidden_size[1], activation='relu'),
        tf.keras.layers.Dense(num_classes)
    ])
    
    if flag is None:
        ## init weights from the fair model
        f = h5py.File('path/to/saved_model', 'r')
        bias = []
        kernel = []
        for i in list(f.keys()):
            bias.append(f[i][i]['bias:0'].value)
            kernel.append(f[i][i]['kernel:0'].value)
        ctr = 0
        for layer in model.layers:
            if ctr == 2:
                layer.set_weights([kernel[ctr][:,:2], bias[ctr][:2]])
            else:
                layer.set_weights([kernel[ctr], bias[ctr]])
            ctr += 1
    return model

#### Training function for fair-SGD

In [ ]:
def train_per_model(train_x, train_y, test_x, test_y, train=True, wts=None):
    
    model = model_init(1)
    
    if wts != None:
        model.set_weights(wts)
        
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

    def fair_loss(y_actual, rounded):
        
        y_true = y_actual[:,:2]
        output = rounded[:,:2]
        loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)
        
        loss1 = loss(y_true, output)
        prot_data = y_actual[:,2:]
        rounded = output
        c0 = kb.sum((rounded[:,1] * (prot_data[:,0])))/ (kb.sum(prot_data[:,0]))
        c1 = kb.sum((rounded[:,1]* prot_data[:,1]))/(kb.sum(prot_data[:,1]))
        loss2 = kb.maximum(kb.abs(c0 - c1) - epsilon, 0.0)
        return loss1 + rho * loss2
    
    def DemP(y_true, y_pred):
        y_pred = kb.softmax(y_pred[:,:2])
        prot_data = y_true[:,2:]
        c0 = kb.sum((y_pred[:,1] * (prot_data[:,0])))/ (kb.sum(prot_data[:,0]))
        c1 = kb.sum((y_pred[:,1]* prot_data[:,1]))/(kb.sum(prot_data[:,1]))
        loss2 = kb.abs(c0 - c1)
        return loss2

    
    if train:
        model.compile(optimizer=optimizer, loss= fair_loss, metrics=['accuracy', DemP])
        es = tf.keras.callbacks.EarlyStopping(monitor='val_acc', mode='max', min_delta=0.01, patience=200)
        model.fit(train_x, train_y,
                  epochs=num_epochs,
                  validation_data=(test_x, test_y),
                  batch_size=batch_size, callbacks=[es])

    else:
        model.set_weights(wts)
        model.compile(optimizer=optimizer, loss=fair_loss, metrics=['accuracy', DemP])
    return model

#### Training function for DP-SGD

In [ ]:
model_target = model_init(None)
def train_per_model(train_x, train_y, test_x, test_y, train=True, wts=None):
    
    model = model_init(0)    
    
    optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
        l2_norm_clip=l2_norm_clip,
        noise_multiplier=noise_multiplier,
        num_microbatches=num_microbatches,
        learning_rate=learning_rate)
    
    if wts != None:
        model.set_weights(wts)

    
    loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.losses.Reduction.NONE)
    
    if train:
        model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
        es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.1, patience=10)
        model.fit(train_x, kb.eval(kb.softmax(model_target(train_x))),
                  epochs=10,
                  validation_data=(test_x, test_y),
                  batch_size=batch_size, callbacks=[es])
    else:
        model.set_weights(wts)
    return model

### Train and test split

In [ ]:
test_ix = np.random.randint(0, len(data), int(0.2*len(data)))
test_data = data[test_ix, :-1]
test_prot =  tf.keras.utils.to_categorical(prot[test_ix, np.newaxis], num_classes=num_classes)
test_labels = tf.keras.utils.to_categorical(data[test_ix, -1], num_classes=num_classes)
test_labels = np.append(test_labels, test_prot, 1)

split_datasets = []
split_labels = []
for i in range(splits):
    t_ix = np.random.randint(0, len(data), samples_per_network)
    split_datasets.append(data[t_ix, :-1])
    split_prot = tf.keras.utils.to_categorical(prot[t_ix, np.newaxis], num_classes=num_classes)
    split_labels.append(np.append(tf.keras.utils.to_categorical(data[t_ix, -1], num_classes=num_classes), split_prot, 1))
    
trained_wts = None
for itr in range(int(num_epochs/10)):
    for i in range(splits):
        model = train_per_model(split_datasets[i], split_labels[i], test_data, test_labels[:,:2], wts = trained_wts)
        if i == 0:
            layerwise_weights = []
            for l in model.get_weights():
                layerwise_weights.append(l/splits)
        else:
            ctr = 0
            for l in model.get_weights():
                layerwise_weights[ctr] += l/splits
                ctr += 1
    trained_wts = layerwise_weights

### Moments accountant

In [ ]:
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=samples_per_network,
                                              batch_size=batch_size,
                                              noise_multiplier=noise_multiplier,
                                              epochs=num_epochs,
                                              delta=1e-4)